In [ ]:
source(here::here("R/setup.R"))

# 数値データの取り扱い

データ分析においてもっとも一般的な型が数値データです。商品の価格やウェブページのアクセス件数、株価の変動など多くのデータが数値で表されます。このことからわかるように、一言で数値データといっても実数から整数、マイナスの値を持つものまで多様な種類が存在します。

多くのモデルが数値の入力を前提としているため、数値をそのまま利用することもできます。しかし特徴量エンジニアリングが不要というわけではありません。具体的には線形回帰モデルでは、出力から得られる値の誤差が正規分布に従うことを仮定します。そのため正規分布とは異なる形状の分布をもつデータ、例えば離散値ではその仮定が成立しない可能性があります。この問題を解決するため、元のデータを正規分布に近似させるという特徴量エンジニアリングが有効になります。

良い特徴量というのはデータの特徴を強く反映します。連続的な数値の二値化あるいは離散化により、モデルの精度を改善できる見込みがあります。また数値以外のテキストや画像データを数値化した際、さらなる特徴量エンジニアリングが必要になることがあります。つまり数値データの処理は特徴量エンジニアリングの中で最も基本的な技と言えます。

前章で示した標準化や正規化も数値データの処理ですが、この章では数値変数をモデルに適した形式へと変換する手法を紹介します。元が数値でないデータの扱い各章で解説します。単一の変数を対象にした処理として対数変換、離散化、ハッシュ化を扱います。また複数の特徴量から新たな特徴量を生成する手法や変数間の相互作用について導入を行います。

## 数値データが抱える問題

数値データはありふれたデータ形式ですが、その反面数多くの問題を抱えていることがあります。その特徴をあげてみます。

- スケールが大きく異なる
- 歪んだ分布をもつ
- 大小の外れ値を含む
- 変数間で、線形では表現できないような複雑な関係を持っている
- 冗長な情報

これらの問題は、回帰か分類かという課題設定に応じて、適用するモデルの種類によって顕在化します。しかし適切なモデルを選択することで問題のいくつかを軽減できる見込みもあります。例を見てみましょう。

k近傍法やサポートベクターマシンは、特徴空間上の外れ値の影響を受けやすい性質があります。一方で、実際の値ではなく順位化されたデータを利用する木ベースのモデルでは外れ値の影響を軽減可能です。また互いに説明変数の間で強い相関がある変数を重回帰モデルに組み込むと、わずかに値が変動しただけで係数が大きく異なってしまいます。このような状況では部分最小二乗法を用いることで説明変数の相関を無相関化できます。つまり、問題のあるデータに対しては適切なモデルとデータの変換を選択することによりある程度の対策が可能なのです。

数値データに対する特徴量エンジニアリングの作戦は次の3種類に分けて考えられます。

- 個々の数値変数の問題について直接対処する
- 変数変換
- 変数間の関係を見つめ直す

この章では、これらの作戦を個別に紹介してきます。

## 対数変換

数値データの特徴量エンジニアリングとして最初に検討するのが対数変換です。これは[前処理](01/tidy-data)で扱ったスケール変換とは対象の変数の値を変えるという意味では同じですが、大きな違いがあります。スケール変換では変数のばらつきを元にする変換のために変換後の値でも分布は変わりません。しかし対数変換ではデータの分布が変化します。

対数変換の効果が大きいのは偏りがある数値データです。裾の長いデータ (long tail) とも呼びます。次の図は地価公示データの地価をヒストグラムにプロットした図と、同じデータに対数変換を施したものです。

In [ ]:
p_base <- 
  df_lp_kanto %>% 
  ggplot(aes(posted_land_price)) +
  geom_histogram(bins = 30, show.legend = FALSE) +
  scale_fill_identity()

p1 <- p_base + 
  aes(fill = ds_col(1)) +
  xlab("円/m^2")
p2 <- p_base + 
  aes(fill = ds_col(5)) +
  xlab("log10(円/m^2)") +
  scale_x_log10()

plot_grid(
  p1,
  p2,
  ncol = 2)

![](../images/log_transform_compared-1.png)

対数変換には次の特徴があります。特徴量のスケールが大きい時はその範囲を縮小し、小さい時は拡大します。これにより、裾の長い分布を押しつぶしたように山のある分布に近づけることができます。また分散が大きなデータでは平均値が大きいほど等分散になりやすい傾向にあります。

### Box-Cox変換

正規分布に従わないデータに対して線形回帰モデルを適用すると推定が思うように行かないことがあります。これは線形回帰モデルが一般に入力に対して正規分布のデータを仮定しているためです。このようなデータでは、正規分布でない分布を背景とする回帰モデルを利用することが一番の対策です。しかしここではデータの分布を変えることでデータを正規分布に近づける方法を紹介しましょう。

ポアソン分布は平均と分散が等しいという性質があります。平均 $\lambda$ が大きいほど分散が大きくなり、分布の裾が重くなります。地価公示データの土地の面積を示す変数 `acreage` は裾の重い分布をしてます (ヒストグラムは後ほど作成します)。 このデータを正規分布に近づけるにはどうすれば良いでしょう。先に紹介した対数変換は1つの手段です。

データが正規分布に従っているかは、シャピロ-ウィルクの正規性検定や正規確率グラフ (QQプロット) の描画で確かめられます。 を例に見てみます。元のデータと対数変換した値を使って、シャピロ-ウィルクの正規性検定を行います。この統計手法の帰無仮説は「変数は正規分布にしたがう」です。 $P < 0.05$ となれば帰無仮説は棄却されます。すなわち正規分布の仮定が保証できなくなります。

In [ ]:
shapiro.test(sample(df_lp_kanto$acreage, 5000)) %>% tidy()
shapiro.test(log(sample(df_lp_kanto$acreage, 5000))) %>% tidy()
shapiro.test(log(sample(df_lp_kanto$acreage, 5000), base = 10)) %>% tidy()

いずれの結果も統計的有意に帰無仮説を棄却するものでした。続いて正規確率グラフを作成します。このグラフでは、度数分布図で得られた度数の累積値より作成される累積度数分布図の縦軸を変形したもので、データが正規分布に従うのであれば直線上に並ぶというものです。

In [ ]:
ggplot(df_lp_kanto, aes(sample = log(acreage, base = 10))) + 
  stat_qq() +
  stat_qq_line()

![](../images/qq_plot-1.png)

対数変換により小さな値の集団は正規分布に近づきましたが、途中から大きくずれてしまいました。これはデータの平均と分散の大きさを反映した結果です。

分散が平均に依存しないように変換をする方法として分散安定変換があります。ポアソン分布に従うデータ $X$ を平方根で変換すると $\tilde{X} = \sqrt{X}$ となり、平均とは無関係に分散がほぼ一定になります。

対数変換と平方根変換を一般化したものとしてBox-Cox変換があります。Box-Cox変換はパラメータ $\lambda$ を指定する必要があり、パラメータ $\lambda$ を変更することで、対数変換 ($\lambda$ = 0)、平方根変換 ($\lambda$ = 0.5)、逆数変換 ($\lambda$ = -1.0) に対応します。

土地の面積にBox-Cox変換を施し、改めて正規性の検定を行いましょう。

In [ ]:
rec_lp_acreage <- 
  recipe(~ acreage, data = df_lp_kanto)

df_lp_acreage_boxcox <- 
  rec_lp_acreage %>% 
  step_BoxCox(all_predictors()) %>%
  prep(training = df_lp_kanto) %>% 
  juice(all_predictors())

In [ ]:
shapiro.test(sample(df_lp_acreage_boxcox$acreage, 5000)) %>% tidy()

In [ ]:
df_lp_acreage_boxcox %>% 
  ggplot(aes(sample = acreage)) +
  stat_qq() +
  stat_qq_line()

統計的には正規分布ではないと判断されましたが、正規確率グラフを見るとより多くのデータが正規分布に近づいていることがわかります。最後に、土地面積のデータにさまざまな変換を施したヒストグラムを比較のために作っておきましょう。

![](../images/qq_plot_after_boxcox_transform-1.png)

In [ ]:
p1 <- 
  ggplot(df_lp_kanto, aes(acreage)) +
  geom_histogram(bins = 30) +
  ggtitle("raw") +
  aes(fill = ds_col(1)) +
  scale_fill_identity()

p2 <- 
  rec_lp_acreage %>%
  step_sqrt(all_predictors()) %>%
  prep(training = df_lp_kanto) %>%
  juice(all_predictors()) %>%
  ggplot(aes(acreage)) +
  geom_histogram(bins = 30) +
  ggtitle("sqrt") +
  aes(fill = ds_col(2)) +
  scale_fill_identity()

p3 <- 
  rec_lp_acreage %>%
  step_log(all_predictors(), base = 10) %>%
  prep(training = df_lp_kanto) %>%
  juice(all_predictors()) %>%
  ggplot(aes(acreage)) +
  geom_histogram(bins = 30) +
  ggtitle("log (base10)") +
  aes(fill = ds_col(3)) +
  scale_fill_identity()

p4 <- 
  df_lp_acreage_boxcox %>% 
  ggplot(aes(acreage)) +
  geom_histogram(bins = 30, show.legend = FALSE) +
  ggtitle("Box-Cox") +
  aes(fill = ds_col(4)) +
  scale_fill_identity()

plot_grid(p1, p2, p3, p4, ncol = 2, labels = "AUTO")

![](../images/lp_acreage_transform-1.png)

対数変換やBox-Cox変換は、0や負値を扱えないという欠点もあります。そのため、強引に正の値となるような加算の処理をしてから対数変換を行うことがあります。このような処理に対して、足し合わせる数値を下駄に例えて「下駄を履かせる」と表現することがあります。ただしこの「下駄」も負値についても非負値とするために「...を足す」のような処理をすると解釈が難しくなる問題があります。ここでは詳しく解説しませんが、0や負値を含んだ特徴量を扱えるYeo-Johnson変換 (Yeo-Johnson Power Transformations) が効果的な時があります。

In [ ]:
# 0 や 負値では対数変換が行えない
log(0)
log10(0)
log10(-1)
# 下駄を履かせる処理
log10(0 + 1)

## ロジット変換

- 百分率や比率データ
    - 0から1の範囲に含まれる値を確率として表現。
    - 変換前と変換後の値を、散布図にすると曲線を描く

In [ ]:
df_lp_kanto %>% skimr::skim()
d <- 
  df_lp_kanto %>% 
  transmute(frontage_ratio = frontage_ratio * 0.01)

rec <- 
  recipe(~ frontage_ratio, data = d)

df_transed <- rec %>% 
  step_logit(all_predictors()) %>% 
  prep(training = d) %>% 
  juice(all_predictors()) %>%
  rename(frontage_ratio_trans = frontage_ratio) %>% 
  bind_cols(d)

df_transed %>% 
  ggplot(aes(frontage_ratio, frontage_ratio_trans)) +
  geom_point()

## ビン詰め (ビニング, binning)

ビン詰めあるいは離散化は、量的変数を1組あるいはそれ以上のカテゴリに変換する作業です。ここで新たに「ビン」という言葉が出てきました。ビンはカテゴリの値が入る容器だと考えてください。ヒストグラムで使われる「ビン」と同じものです。数値データを扱う際、単一の変数の四分位数を求めることがありますが、それと同じような作業です。四分在数ではデータのばらつきを3つに区切ります。

- 結果の解釈が簡単になる
- データの変動を抑える

いろいろな方法がある

一変数統計量として


連続値をとる数値を離散値に

線形モデルで効果を発揮します。

複数の特徴量に分割する

決定木では利点がない

- スケールによる影響を緩和するためにカウントデータを離散化

地上階数はカウントデータとして与えられています。

カウントした値を階級にまとめる方法です。

階級データには順序が与えられることになります。

### 固定幅による離散化

- 各階級の範囲をあらかじめ決めておく
- 階級区分に規則性がある場合とない場合
    - 年齢... 10歳区切り。階級幅は一定で規則性がある
    - 年齢... ライフスタイルによる区切り。規則性がない
- 10の累乗 (0~9, 10~99, 100~9999) 指数関数的

### 分位数による離散化

- 均等に

In [ ]:
# 5段階の階級を設定
binned <- 
  discretize(df_lp_kanto$distance_from_station, 
             cuts = 5, 
             infs = FALSE, 
             keep_na = FALSE,
             prefix = "distance_bins")
table(predict(binned, df_lp_kanto$distance_from_station))

## シークエンス

- 移動平均
- 外れ値をもつ場合には平滑化が有効

In [ ]:
mod_rec <- 
  df_lp_kanto %>%
  select(posted_land_price, number_of_floors) %>% 
  recipe(formula = ~ .)

df_trans <- 
  mod_rec %>% 
  step_BoxCox(all_numeric()) %>% 
  prep() %>% 
  juice()

p1 <- df_lp_kanto %>% 
  ggplot(aes(posted_land_price)) +
  geom_histogram(bins = 30, fill = ds_col(1))

p2 <- df_trans %>% 
  ggplot(aes(posted_land_price)) +
  geom_histogram(bins = 30, fill = ds_col(5))

plot_grid(p1, p2)

## カウントデータの取り扱い

## 二値化

In [ ]:
# box-cox変換 ---------------------------------------------------------------
# モデルを定義... 線形回帰モデル
spec_lin_reg <- linear_reg()
# エンジンを指定... stats::lm
spec_lm <- set_engine(spec_lin_reg, engine = "lm")


# 同じ結果... formulaで参照するのではなく、x,yにデータを与える
fit_xy(spec_lm, 
       y = df_train_baked %>% pull(posted_land_price),
       x = df_train_baked %>% 
         select(-posted_land_price)) %>% 
  tidy()

df_train_price_log <-
  df_train %>%
  mutate(price = log10(posted_land_price),
         distance_from_station = log10(distance_from_station + 1))

# df_train %>% 
#   recipe(mod_formula) %>% 
#   step_BoxCox(all_outcomes(), lambdas = 1.0) %>% 
#   step_log(distance_from_station, base = 10) %>% 
#   prep(training = df_train) %>% 
#   juice()

# stanエンジン (rstan::stan())
spec_stan <- 
  spec_lin_reg %>% 
  set_engine(engine = "stan", chains = 4, iter = 1000)

fit_stan <- fit(
  spec_stan,
  mod_formula2,
  data = df_train)

coef(fit_stan$fit)
fit_stan %>% broom::tidy()

# knn
fit_knn <-
  nearest_neighbor(mode = "regression", neighbors = 4) %>% 
  set_engine("kknn") %>% 
  fit(mod_formula2, data = df_train)

predict(fit_knn, new_data = df_test %>% 
          select(-posted_land_price))

car::powerTransform(df_lp_kanto$posted_land_price)

library(car)
summary(p1 <- powerTransform(cycles ~ len + amp + load, data = Wool))
# fit linear model with transformed response:
coef(p1, round=TRUE)
summary(m1 <- lm(bcPower(cycles, p1$roundlam) ~ len + amp + load, Wool))

In [ ]:
df_lp_kanto %>% 
  count(current_use) %>% 
  ggplot(aes(forcats::fct_reorder(current_use, n), n)) +
  geom_bar(stat = "identity") +
  coord_flip()
df_lp_kanto %>% 
  count(use_district) %>% 
  ggplot(aes(forcats::fct_reorder(use_district, n), n)) +
  geom_bar(stat = "identity") +
  coord_flip()

recipe(formula = mod_formula3, data = df_train)

mod_rec <- 
  recipe(formula = mod_formula3, data = df_train) %>% 
  step_log(all_outcomes(), base = 10) %>% 
  # 5%未満を "other"
  step_other(use_district, threshold = 0.05) %>% 
  step_dummy(all_nominal())

mod_rec

# recipe (define) -> prep (calculate) -> bake/juice (apply)
mod_rec_trained <- prep(mod_rec, training = df_train, verbose = TRUE)

lp_test_dummy <- bake(mod_rec_trained, new_data = df_test)
names(lp_test_dummy) # 1住居がない（一番多い）

In [ ]:
# interaction
p <- ggplot(df_train,
       aes(distance_from_station, posted_land_price)) +
  geom_point() +
  scale_y_log10()

p +
  geom_smooth(method = "loess")

# MASS::rlm()
library(MASS)
p + 
  facet_wrap(~ gas_facility, ncol = 1) +
  geom_smooth(method = "rlm")

mod1 <- lm(log10(posted_land_price) ~ distance_from_station, data = df_train)
mod2 <- lm(log10(posted_land_price) ~ distance_from_station + gas_facility + distance_from_station:gas_facility , data = df_train)

anova(mod1, mod2)

mod_formula4 <- formula(posted_land_price ~ distance_from_station + gas_facility)
df_train %>% 
  recipe(mod_formula4, data = .) %>% 
  step_log(all_outcomes()) %>% 
  step_dummy(gas_facility) %>% 
  step_interact(~ starts_with("gas_facility"):distance_from_station) %>% 
  prep(training = df_train) %>% 
  juice()

<!-- ビニングや多項式特徴量、-->

## より豊かな表現

モデルの表現を豊かにするための特徴量エンジニアリングとして

多項式特徴量や交互作用特徴量が利用可能です。

線形モデルで有効?

特に線形モデルでは一般に多項式特徴量、交互作用特徴量を追加することで精度向上が期待されます。



### 多項式特徴量

In [ ]:
data(biomass)

biomass_tr <- biomass[biomass$dataset == "Training",]
biomass_te <- biomass[biomass$dataset == "Testing",]

rec <- recipe(HHV ~ carbon + hydrogen + oxygen + nitrogen + sulfur,
              data = biomass_tr)

quadratic <- rec %>%
  step_poly(carbon, hydrogen)
quadratic <- prep(quadratic, training = biomass_tr)

expanded <- bake(quadratic, biomass_te)
expanded


### 交互作用特徴量

## 次元集約によると特徴量の作成

- PCA, ICA
- L2ノルム正則化... ベクトル空間に対して「距離」を与えるための数学の道具

過学習を防ぐためでもある

<!-- reduction... ここでは概要説明のみ。詳しくは該当する章で-->

## まとめ

> 対数をとるモデルと対数をとらないモデルのどちらか一方が「正しい」わけではない。あくまでも「そういう仮定を選んだ」ということに過ぎない

数値特徴量

特徴量の管理、膨大に増えた特徴量の次元削減については別の章で解説します。

## 関連項目

- [特徴量選択](feature-selection)

## 参考文献

- Max Kuhn and Kjell Johnson (2013). Applied Predictive Modeling. (Springer)
- Sarah Guido and Andreas Müller (2016). Introduction to Machine Learning with Python A Guide for Data Scientists (O'Reilly) (**翻訳** 中田秀基訳 (2017). Pythonではじめる機械学習 scikit-learnで学ぶ特徴量エンジニアリングと機械学習の基礎 (オライリー))
- Max Kuhn and Kjell Johnson (2019). Feature Engineering and Selection: A Practical Approach for Predictive Models (CRC Press)